In [1]:
Chart = list(range(1, 26))
Cli = [*list(range(1, 6)),*list(range(7,41))]
Closure = [*list(range(1, 63)),*list(range(64,93)),*list(range(94,177))]
Codec = list(range(1, 19))
Collections = list(range(25, 29))
Compress = list(range(1, 48))
Csv = list(range(1, 17))
Gson = list(range(1, 19))
JacksonCore = list(range(1, 27))
JacksonDatabind = list(range(1, 113))
JacksonXml = list(range(1, 7))
Jsoup = list(range(1, 94))
JxPath = list(range(1, 23))
Lang = [*list(range(1, 2)),*list(range(3,66))]
Math = list(range(1, 107))
Mockito = list(range(1, 39))
Time =  [*list(range(1, 21)),*list(range(22,28))]

# Define the range, based on the parameters above
# Define the project, based on Defects4J nomenclature
range = Csv
project = "Csv"

In [2]:
# This notebook generates an ochiai ranking for all the valid versions of CSV defects4j project
# The final csv file can be use to analyse whether line of code was found on the first positions of ranking
# We can check the score, ranking position and filter constrained and unconstrained duas

# Input/Output folders setup section:
# Location of files (csv and json) generated by the runFaultDetect.sh script
global CONST_RANKING_FILES_FOLDER
global CONST_PARSED_FILES_FOLDER
global CONST_BUGGY_LINES_FOLDER
global CONST_RANKING_FOLDER
global CONST_PROJ_RANKING_FOLDER

CONST_RANKING_FILES_FOLDER = "D:/dataflow/ranking/" + project + "/"
CONST_PARSED_FILES_FOLDER = "D:/experiment/parsedRanking/"
CONST_BUGGY_LINES_FOLDER = "D:/dataflow/buggyLines/" + project + "/"
CONST_RANKING_FOLDER = "D:/experiment/ochiai/ranking/"
CONST_PROJ_RANKING_FOLDER = "D:/experiment/ochiai/ranking/" + project + "/"
CONST_PROJ_CLASSES_FOLDER = "D:/dataflow/classes/" + project + "/"

import os
try:
    os.remove(CONST_PROJ_RANKING_FOLDER)
    os.remove(CONST_PARSED_FILES_FOLDER)
    print("% s Diretorio removido com sucesso")
except OSError as error:
    print(error)
    print("Diretorio nao existe")
    
os.makedirs(CONST_PROJ_RANKING_FOLDER)
os.makedirs(CONST_PARSED_FILES_FOLDER)

import re
import pandas as pd

# Loop on the range of valid CSV ranking versions and edit of the content
# replacing #,@ and $ delimiters by the comma delimiter
#for i in range(1,17, 1):
for i in list(range):
    csv = CONST_RANKING_FILES_FOLDER + str(i) + "b_RankingDuas.csv"
    data = pd.read_csv(csv, delimiter=';')
    df1 = pd.DataFrame(data)
    df2 = df1["DUAs"]
    cols = [3,4,5]

    df1.drop(df1.columns[cols],axis=1,inplace=True)
    df2 = df2.str.split(pat=r"%|#|@", expand=True)
    df3 = df2[3]
    df3 = df3.str.split(pat=r"\$", expand=True, n=1)

    df2.drop(df2.columns[3],axis=1,inplace=True)
    df3.rename(columns = {0: 'dua#', 1: 'dua'}, inplace = True)
    df = pd.concat([df1,df2,df3], axis=1, join='inner')
    df.rename(columns = {'OCHIAI   \t': 'Score', 'Uncon': 'uncon', 'Status': 'status', 0: 'class', 1: 'method', 2: 'method#'}, inplace = True)

    csv=CONST_PARSED_FILES_FOLDER + project + "_" + str(i) + "b_Ranking.csv"
    # Generation of comma delimited ranking files
    df.to_csv(csv)

[WinError 3] O sistema não pode encontrar o caminho especificado: 'D:/experiment/ochiai/ranking/Csv/'
Diretorio nao existe


In [3]:
# Loop on the range of valid CSV buggy.lines files and creation of a standard csv file
# replacing the # delimiter by the comma delimiter, replacing the package separator and
# removing the java file extension

#for i in range(1,17, 1):
for i in list(range):
    csv = CONST_BUGGY_LINES_FOLDER + project + "-"+str(i)+".buggy.lines"
    data = pd.read_csv(csv, delimiter='#',header=None)
    data.rename(columns={data.columns[0]: 'class', data.columns[1]: 'line',data.columns[2]: 'code'}, inplace = True)
    csv = CONST_BUGGY_LINES_FOLDER + project + "-"+str(i)+".buggy.lines.csv"
    data["class"] = data["class"].apply(lambda x: x.replace("/", "."))
    data["class"] = data["class"].apply(lambda x: x.replace(".java", ""))
    data.to_csv(csv)

In [4]:
# Loop on the range of valid CSV buggy.lines and parsed rankings in order to merge information of both sources
# and detect whether line of code is part of duas being ranked

#for i in range(1,17, 1):
for i in list(range):
    csv = CONST_BUGGY_LINES_FOLDER + project + "-"+str(i)+".buggy.lines.csv"
    data1 = pd.read_csv(csv, index_col=0, delimiter=',')

    csv = CONST_PARSED_FILES_FOLDER + project + "_"+str(i)+"b_Ranking.csv"
    data2 = pd.read_csv(csv, delimiter=',')
    data2.rename(columns={'Unnamed: 0': 'Ranking'}, inplace = True)

    merged_inner = pd.merge(left=data2, right=data1, left_on='class', right_on='class')
    merged_inner['LoC'] =  [re.findall(r'\b\d+\b', str(x)) for x in merged_inner['dua']]
    merged_inner['line'] = merged_inner['line'].astype(str)
    merged_inner['LoC'] = merged_inner['LoC'].astype(str)

    # Search of the bug line of code in the dua specification and population of the found boolean column    
    found = []
    for j, row in merged_inner.iterrows():
        lineOfCode = "'" + row.line + "'"
        if lineOfCode in row.LoC:
            found.append(True)
        else:
            found.append(False)
    merged_inner['found']=found

    # Generation of Ranking for each version
    csv = CONST_PROJ_RANKING_FOLDER + project + "_" +str(i)+ "_Result.ranking.csv"
    merged_inner.to_csv(csv, sep=';')
    
    # Generation of Relevant Ranking for each version
    colnames=['class'] 
    csv = CONST_PROJ_CLASSES_FOLDER + str(i) + "b/"+"classes.relevant"
    relevant=pd.read_csv(csv, delimiter=';', names=colnames, header=None)
    relevant.replace('src/main/java/', '', regex=True, inplace=True)
    relevant.replace('/', '.', regex=True, inplace=True)
    merged_inner_final = pd.merge(left=merged_inner, right=relevant, left_on='class', right_on='class')
    csv = CONST_PROJ_RANKING_FOLDER + project + "_" +str(i)+ "_Relevant_Result.ranking.csv"
    merged_inner_final.to_csv(csv, sep=';')

In [5]:
# Loop on the range of valid classes.modified and classes.relevant files in order to
# remove the java default package folder of the package names and replace the package separator
#for i in range(1,17, 1):
for i in list(range):
    colnames=['class'] 
    csv = CONST_PROJ_CLASSES_FOLDER + str(i) + "b/"+"classes.relevant"
    relevant=pd.read_csv(csv, delimiter=';', names=colnames, header=None)
    relevant.replace('src/main/java/', '', regex=True, inplace=True)
    relevant.replace('/', '.', regex=True, inplace=True)

In [6]:
# Generation of the Ranking for each of the versions and a final unified ranking containing all the versions
li = []
#for i in range(1,17, 1):
for i in list(range):
    csv = CONST_PROJ_RANKING_FOLDER + project + "_" +str(i)+ "_Relevant_Result.ranking.csv"
    data=pd.read_csv(csv, delimiter=';')
    data['version'] = str(i) + "b"
    li.append(data)
frame = pd.concat(li, axis=0, ignore_index=True)
csv = CONST_RANKING_FOLDER + project + "_FinalRanking.csv"
frame.to_csv(csv, sep=';')